# CNMF-E pipeline for 1p
This notebook demonstrates how to use Caiman for processing 1p microendoscopic data. It shows how to use Caiman for the following steps:

![workflow diagram](images/full_cnmfe_workflow.jpg)

1. Apply the nonrigid motion correction (NoRMCorre) algorithm for motion correction to the original movie.
2. Apply the constrained nonnegative matrix factorization endoscopic (CNMF-E) source separation algorithm to extract an initial estimate of neuronal spatial footprint and calcium traces.
3. Apply quality control metrics to evaluate the initial estimates to narrow them down to a final set of estimates.

Tools for visualization of movies and results are also included.

> This demo follows a similar pattern to the CNMF demo in `demo_pipeline.ipynb`. It includes less explanation except where there are important differences. If you want to get a more explanation-heavy picture of the fundamentals, we suggest starting with `demo_pipeline.ipynb`.

# Imports and general setup

In [ ]:
import bokeh.plotting as bpl
import cv2
import glob
import holoviews as hv
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil
import scipy.io
import scipy.sparse
import pandas as pd
import scipy
from pathlib import Path
from scipy.interpolate import interp1d

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.utils.visualization import view_quilt

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
        # get_ipython().run_line_magic('matplotlib', 'qt')  #uncomment to run in qt mode
except NameError:
    pass

try:
    cv2.setNumThreads(0)
except:
    pass

bpl.output_notebook()
hv.notebook_extension('bokeh')

# Set play_movies to False if you want to disable play of movies, e.g. for remote-hosted Jupyter environments
play_movies = False # 260112 set to false

Set up logger and environment variables

In [ ]:
# set up logging
logfile = None # Replace with a path if you want to log to a file
logger = logging.getLogger('caiman')
# Set to logging.INFO if you want much output, potentially much more output
logger.setLevel(logging.WARNING)
logfmt = logging.Formatter('%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s')
if logfile is not None:
    handler = logging.FileHandler(logfile)
else:
    handler = logging.StreamHandler()
handler.setFormatter(logfmt)
logger.addHandler(handler)

# set env variables in case they weren't already set
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"

# Set path to file to be processed
In this demo, we analyze the data in `data_endoscope.tif`. This movie provides sample microendoscopic data from mouse dorsal striatum where the neurons are expressing GCaMP6f. The original movie was sampled at 30Hz but spatially cropped and downsampled to 10Hz to save space for the demo -- see [the original CNMFE paper](https://elifesciences.org/articles/28728#s3) for more details about the data.

The `download_demo` function will download the file and return the filepath, which will be stored in your `caiman_data` directory:

In [ ]:
# Edit the path
base_dir = r"C:\Personal Drive\Undergraduate\2026 Winter\miniscope\MY01_20251231\My_V4_Miniscope"
movie_path = [os.path.join(base_dir, f"{i}.avi") for i in range(22)]

> When adapting this demo for your data make sure to store the full path to your movie in the `movie_path` variable. Also, for reasons we will explain below, the memory requirement of the CNMF-E algorithm are much higher compared to the CNMF algorithm. If your datasets are too large, you might need to use our online algorithms. We discuss this more in `demo_pipeline.ipynb`.

# Load and visualize raw data
We visualize using the built-in movie object, which is described in detail in `demo_pipeline.ipynb`. In addition to neural activity, you can also see blood flow through blood vessels in the movie. Such background activity, standard for 1p data, brings new analysis challenges.

In [ ]:
# press q to close
movie_orig = cm.load(movie_path)
downsampling_ratio = 0.2  # subsample 5x
movie_orig.resize(fz=downsampling_ratio)
if play_movies:
    movie_orig.play(gain=0.9,
                    q_max=99.5,
                    fr=10,
                    plot_text=True,
                    magnification=4,
                    do_loop=True,
                    backend='opencv')

# Set up multicore processing
To enable parallel computing we will enable multiple CPUs for processing. The resulting `cluster` variable contains the pool of processors (CPUs) that will be used in later steps. If you use `dview=cluster` in later steps, then parallel processing will be used. If you use `dview=None` then no parallel processing will be used. The `num_processors_to_use` variable determines how many CPU cores you will use (when set to `None` it goes to the default of one less than the number available):

In [ ]:
print(f"You have {psutil.cpu_count()} CPUs available in your current environment")
num_processors_to_use = None

If you have already set up multiprocessing (the `cluster` variable is already in your namespace), then that cluster will be closed and a new one created.

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'cluster' in locals():  # 'locals' contains list of current local variables
    print('Closing previous cluster')
    cm.stop_server(dview=cluster)
print("Setting up new cluster")
_, cluster, n_processes = cm.cluster.setup_cluster(backend='multiprocessing',
                                                 n_processes=num_processors_to_use,
                                                 ignore_preexisting=False)
print(f"Successfully set up cluster with {n_processes} processes")

# Define parameters
We first set some parameters related to the data and motion correction and create a `params` object. We'll modify this parameter object later on with settings for source extraction. You can also set all the parameters at once as demonstrated in the `demo_pipeline.ipynb` notebook.

Note here we are setting `pw_rigid` to `False`, as our data seems to mainly contain large-scale translational motion. We can always redo this later if it turns out to be a mistake.

In [ ]:
# dataset dependent parameters
frate = 20                       # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = False         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # sigma for high pass spatial filter applied before motion correction, used in 1p data
max_shifts = (5, 5)      # maximum allowed rigid shift
strides = (48, 48)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)      # overlap between patches (size of patch = strides + overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries

mc_dict = {
    'fnames': movie_path,
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}

parameters = params.CNMFParams(params_dict=mc_dict)

# Motion Correction
The background signal in micro-endoscopic data is very strong and makes motion correction challenging. As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the lower-frequency background activity and enhance spatial landmarks. The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no preprocessing is performed (default option, used in 2p data for CNMF).

After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data, not the preprocessed data, so no information is lost before source separation. The motion corrected files are saved in memory mapped format. If no motion correction is performed (i.e., `motion_correct` was set to `False`), then the file gets directly memory mapped.

> For a more detailed exploration of Caiman's motion correction pipeline, see `demo_motion_correction.ipynb`.

The following also plots the discovered displacements in x- and y- directions.

In [ ]:
%%time
if motion_correct:
    # do motion correction rigid
    mot_correct = MotionCorrect(movie_path, dview=cluster, **parameters.get_group('motion'))
    mot_correct.motion_correct(save_movie=True)
    fname_mc = mot_correct.fname_tot_els if pw_rigid else mot_correct.fname_tot_rig
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mot_correct.x_shifts_els)),
                                     np.max(np.abs(mot_correct.y_shifts_els)))).astype(int)
    else:
        bord_px = np.ceil(np.max(np.abs(mot_correct.shifts_rig))).astype(int)
        # Plot shifts
        plt.plot(mot_correct.shifts_rig)  # % plot rigid shifts
        plt.legend(['x shifts', 'y shifts'])
        plt.xlabel('frames')
        plt.ylabel('pixels')
        plt.gcf().set_size_inches(6,3)

    bord_px = 0 if border_nan == 'copy' else bord_px
    fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
else:  # if no motion correction just memory map the file
    fname_new = cm.save_memmap(movie_path, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)

Compare original (left panel) and motion corrected movie (right panel). You will notice they look similar, as there wasn't much motion to begin with. You can see from the shift plot (plotted above) that the extracted shifts were all very small.

In [ ]:
movie_corrected = cm.load(mot_correct.mmap_file) # load motion corrected movie
ds_ratio = 0.2
h = cm.concatenate([movie_orig.resize(1, 1, ds_ratio) - mot_correct.min_mov*mot_correct.nonneg_movie,
                movie_corrected.resize(1, 1, ds_ratio)],
                axis=2)
if play_movies:
    h.play(fr=10, gain=0.9, magnification=2)

## Load memory mapped file
Memory mapping is discussed in more detail in `demo_pipeline.ipynb`.

In [ ]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

# Parameter setting for CNMF-E
Everything is now set up to run source extraction with CNMFE. We will construct a new parameter dictionary and use this to modify the *existing* `parameters` object, using the `change_params()` method.

There are *two* main differences between the CNMF and CNMFE source separation algorithms. The first is the background model (this is discussed in the sidebar below on the Ring Model). The second difference is in how the models are initialized. This is addressed below when we go over setting corr/pnr thresholds for initialization, which we did not have to do for our 2p data.

For now, note that we have set `gnb` to `0`: this is effectively the flag telling Caiman to use CNMFE instead of CNMF.

In [ ]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None for CNMFE
gSig = np.array([3, 3])  # expected half-width of neurons in pixels
gSiz = 2*gSig + 1     # half-width of bounding box created around neurons during initialization
merge_thr = .7      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
tsub = 2            # downsampling factor in time for initialization, increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization, increase if you have memory problems
gnb = 0             # number of background components (rank) if positive, set to 0 for CNMFE
low_rank_background = None  # None leaves background of each patch intact (use True if gnb>0)
nb_patch = 0        # number of background components (rank) per patch (0 for CNMFE)
min_corr = .8       # min peak value from correlation image
min_pnr = 10        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background (increase to 2 if slow)
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor

parameters.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # True for 1p
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px});                # number of pixels to not consider in the borders)

Initialize the model using the parameters.

In [ ]:
cnmfe_model = cnmf.CNMF(n_processes=n_processes,
                        dview=cluster,
                        params=parameters)

<div class="alert alert-info">
    <h2 >CNMF-E: The Ring Model</h2>
   Background activity is very ill-behaved with 1p recordings: it often fluctuates locally and is much larger in magnitude than the neural signals we want to extract. In other words, the large-scale background model used for CNMF is not sufficient for most 1p data. Hence, Pengcheng Zhou and others came up with a localized model of background activity for CNMFE: the background at each pixel is represented as the weighted sum of activity from a circle (or ring) of pixels a certain distance from that pixel. The distance of this ring from the reference pixel is set by the <em>ring_size_factor</em> parameter. This more complex pixel-by-pixel background model explains why CNMFE is computationally more expensive than CNMF, and also why it works better to mop up large-scale localized background noise in  1p data.
    
<p>When you set <em>gnb</em> in the CNMF model (usually to 1 or 2), you are setting the number of global background components. The fact that you can get away with so few is testament to how well-behaved the background activity is in 2p recordings. When we set <em>gnb</em> to 0 in Caiman, this is a flag telling Caiman's back end to switch to the more complicated ring model of the background activity.</p>

For more details on CNMFE you can see the <a href="https://elifesciences.org/articles/28728">original paper</a> and the <a href="https://elifesciences.org/articles/38173">Caiman paper</a>.
</div>

## Key parameters for CNMFE
The key parameters for CNMFE are slightly different than for CNMF, but with some overlap. As we'll see, because of the high levels of background activity, we can't initialize the same way as with CNMF. We have two new important parameters directly related to initialization that come into play: `min_corr` and `min_pnr`.

`rf` (int): *patch half-width*
> `rf`, which stands for 'receptive field', is the half width of patches in pixels. The patch width is `2*rf + 1`. `rf` should be *at least* 3-4 times larger than the observed neuron diameter. The larger the patch size, the less parallelization will be used by Caiman. If `rf` is set to `None`, then CNMFE will be run on the entire field of view.

`stride` (int): *patch overlap*
> `stride` is the overlap between patches in pixels (the actual overlap is `stride_cnmf + 1`). This should be at least the diameter of a neuron. The larger the overlap, the greater the computational load, but the results will be more accurate when stitching together results from different patches. This param should probably have been called 'overlap' instead of 'stride'.

`gSig (int, int)`: *half-width of neurons*
> `gSig` is roughly the half-width of neurons in your movie in pixels (height, width). It is the standard deviation of the mean-centered Gaussian used to filter the movie before initialization for CNMFE. It is related to the `gSiz` parameter, which is the width of the entire kernel filter.

`merge_thr (float)`: *merge threshold*
> If the correlation between two spatially overlapping components is above `merge_thr`, they will be merged into one component.

`min_corr` (float): *minimum correlation*
> Pixels from neurons tend to be correlated with their neighbors. During initialization, Caiman filters out those pixels below `min_corr` to help select seed pixels. We discuss this more below.

`min_pnr` (float): *minimum peak to noise ratio*
> Pixels from neurons tend to have a high signal-to-noise ratio. During initialization, Caiman filters out those pixels below `min_pnr` to help select seed pixels. We discuss this more below.

## Inspect summary images and set parameters
### Correlation-pnr plot
For CNMFE, Caiman uses the correlation and peak-to-noise ratio (PNR) for initialization, which will both tend to be high in regions that contain neurons. Hence, we set a threshold for both quantitites to remove the low correlation/low pnr regions and seed initialization with regions most likely to contain neuronal activity.

First, we calculate the correlation and pnr maps of the raw motion corrected movie after filtering with a mean-centered Gaussian with standard deviation `gSig` (for more information on this preprocessing step, see the sidebar below on CNMFE initialization). These calculation can be computationally and memory demanding for large datasets, so we subsample if there are many thousands of frames:

In [ ]:
print(gSig)
gsig_tmp = (3,3)
correlation_image, peak_to_noise_ratio = cm.summary_images.correlation_pnr(images[::max(T//1000, 1)], # subsample if needed
                                                                           gSig=gsig_tmp[0], # used for filter
                                                                           swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile

<img src="images/bokeh_menu.jpg" align="right" width="200"></img>
Using `nb_inspect_correlation_pnr()`, you can inspect the correlation and PNR images to find threshold values for these quantities: `min_corr` and `min_pnr`. You can adjust the range of values displayed in the two subplots below by choosing the Y-box select tool (third button from the left -- highlighted in yellow in the accompanying image to the right) and selecting the desired region in the histograms on the right hand side of each image. You can also use the pan button (first button on the left) to zoom/adjust the axis limits in the histogram to make it easier to see the limits.

In [ ]:
nb_inspect_correlation_pnr(correlation_image, peak_to_noise_ratio, cmap='inferno') # jet, fire are also good cmaps

We are looking for a couple of things in the above plot:
1) Did we filter with a `gSig` value small enough so that we aren't blending different neurons together? To see what it is like when `gSig` is too large, set `gsig_tmp` to `(6,6)` in the above cell and then inspect the resulting correlation-pnr plots.
2) More importantly, we want to find the threshold correlation and pnr values so that the *lower* threshold eliminates most of the noise and blood vessels from the plots, leaving behind as many of the *neural* pixels as possible. For this data it will be at a correlation value lower bound between 0.8 and 0.9, and a pnr lower bound somewhere between 10 and 20. As with CNMF, there is no perfect value. Just keep in mind it is better to have false positives later than false negatives.

You can tweak the parameters in the following cell, using the `change_params()` method:

In [ ]:
gsig_new = gSig # unchanged
min_corr_new  = 0.9 #default: 0.85
min_pnr_new = 10 #default:12

cnmfe_model.params.change_params(params_dict={'gSig': gsig_new,
                                              'min_corr': min_corr_new,
                                              'min_pnr': min_pnr_new});

> Caiman also includes a Qt-based corr-pnr viewer that some people prefer: `inspect_correlation_pnr()`. It provides what some say is a more intuitive interface than the notebook version above. It requires you to be in Qt mode (which you can enable using the cell magic `%matplotlib qt`).

<div class="alert alert-info">
    <h2>CNMFE initialization: More on correlation and peak-to-noise-ratio</h2>
     <img src="images/mn_centered_gaussian.jpg" align="right" width="200"></img>
How are correlation and peak-to-noise ratio actually calculated? First Caiman convolves the motion corrected movie with a <i>mean-centered Gaussian</i> (example to the right). The sigma of the Gaussian is <em>gSig</em>, and mean centering is turned on by setting <em>center_psf</em> to <em>True</em>. This mean centering creates a Gaussian with a positive peak in the middle of width <i>approximately</i> <em>gSig/2</em>, surrounded by a negative trench, and a ledge of zeros around the outer edges. This crucial preprocessing filter serves to highlight neuronal peaks and smooth away low-frequency background activity.

<p>The function <em>correlation_pnr()</em> applies this mean-centered Gaussian to each frame of the motion corrected movie and returns the correlation image of that movie, as well as the peak-to-noise-ratio (PNR). The correlation image is the correlation of each pixel with its neighbors. The PNR is the ratio of the maximum magnitude at a pixel to the noise value at that pixel (it is a fast and rough measure of signal-to-noise). As mentioned above, both of these values tend to be higher in pixels that contain neurons. The CNMFE initialization procedure is to set a threshold for both quantities, take their <i>product</i>, and use the peaks in this product map to find <b>seed pixels</b> for initialization of the CNMFE source separation algorithm.</p>

More details on the initialization procedure used here can be found in the <a href="https://elifesciences.org/articles/28728">CNMFE paper</a>, or by exploring the code.         
</div>

### Evaluate our spatial parameters
As discussed in `demo_pipeline.ipynb`, the other important parameters are those used for dividing the movie into patches for parallelization of the algorithm. Namely, we want to select `rf` and `stride` parameters so that at least 3-4 neuron diameters can fit into each patch, and at least one neuron fits in the overlap region between patches. You can visualize the patches using the `view_quilt()` function:

In [ ]:
# calculate stride and overlap from parameters
cnmfe_patch_width = cnmfe_model.params.patch['rf']*2 + 1
cnmfe_patch_overlap = cnmfe_model.params.patch['stride'] + 1
cnmfe_patch_stride = cnmfe_patch_width - cnmfe_patch_overlap
print(f'Patch width: {cnmfe_patch_width} , Stride: {cnmfe_patch_stride}, Overlap: {cnmfe_patch_overlap}');

# plot the patches
patch_ax = view_quilt(correlation_image,
                      cnmfe_patch_stride,
                      cnmfe_patch_overlap,
                      vmin=np.percentile(np.ravel(correlation_image), 50),
                      vmax=np.percentile(np.ravel(correlation_image), 99.5),
                      color='yellow',
                      figsize=(4,4));
patch_ax.set_title(f'CNMFE Patch Width {cnmfe_patch_width}, Overlap {cnmfe_patch_overlap}');

These patches and overlaps may seem a bit large, but that is ok: our main concern is that they not be too small. If you wanted to change them, you could use `change_params()`.

Now that we are happy with our parameters, let's run the algorithm.

## Run the CNMF-E algorithm

In [ ]:
%%time
cnmfe_model.fit(images);

Running the algorithm creates an `estimates` class, which we discuss in detail in `demo_pipeline.ipynb`. The CNMFE `estimates` class includes almost all the same attributes as with CNMF, such as the neural spatial and temporal components `A` and `C`.

It also includes the discovered model of background activity, which is different from the CNMF model. For *CNMF* the background model is returned as low-rank matrices `b` and `f`. For CNMFE, the background model parameters are represented in the matrix `W` (the weights of the *ring model* -- discussed above -- for each pixel) as well as `b0` (a constant offset for each pixel). We will show how to reconstruct the background activity below.

# Component Evaluation
Source extraction typically produces many false positives. Our next step is quality control: separating the results into "good" and "bad" neurons using two different metrics (discussed in detail in `demo_notebook.ipynb`):

- **Signal-to-noise ratio (SNR)**: a minimum SNR is set for the calcium transients (`min_SNR`).
- **Spatial correlation**:  a minimum correlation is set between the shape of each component and the frames in the movie when that component is active (`rval_thr`).

> Caiman does *not* use the CNN classifier to sort neurons based on shape for 1p data: the network was trained on 2p data. Hence, we set the `use_cnn` param to `False`.

Here we set the two parameters and run `evaluate_components()` to see which pass muster:

In [ ]:
min_SNR = 3            # SNR threshold
rval_thr = 0.85    # spatial correlation threshold

quality_params = {'min_SNR': min_SNR,
                  'rval_thr': rval_thr,
                  'use_cnn': False}
cnmfe_model.params.change_params(params_dict=quality_params)

cnmfe_model.estimates.evaluate_components(images, cnmfe_model.params, dview=cluster)

print('*****')
print(f"Total number of components: {len(cnmfe_model.estimates.C)}")
print(f"Number accepted: {len(cnmfe_model.estimates.idx_components)}")
print(f"Number rejected: {len(cnmfe_model.estimates.idx_components_bad)}")

# Visualize results

In [ ]:
#%% plot contour plots of accepted and rejected components
cnmfe_model.estimates.plot_contours_nb(img=correlation_image,
                                       idx=cnmfe_model.estimates.idx_components);

View traces of accepted and rejected components:

In [ ]:
cnmfe_model.estimates.nb_view_components(img=correlation_image,
                                        idx=cnmfe_model.estimates.idx_components,
                                        cmap='viridis', #gray
                                        thr=.9); #increase to see full footprint

In [ ]:
# rejected components
cnmfe_model.estimates.nb_view_components(img=correlation_image,
                                        idx=cnmfe_model.estimates.idx_components_bad,
                                        cmap='viridis', #gray
                                        denoised_color='red',
                                        thr=0.9); #increase to see full footprint

# A few loose ends
We have extracted the calcium traces C, spatial footprints A, and estimated spike counts S, which is the main goal with CNMF. But there are a few important things remaining.

## Extract $\Delta F/F$ values
Currently in Caiman, we don't return a true dff value for 1p data. This is because, as mentioned in `demo_pipeline.ipynb`,  Caiman normalizes to both the baseline fluorescence and background activity. The background activity in 1p is so ill-behaved (as discussed above in the sidebar on the ring model) that Caiman currently only *detrends* the data by subtracting away the baseline but not normalizing the data. This explains the warning you will see when you run the following:

In [ ]:
if cnmfe_model.estimates.F_dff is None:
    print('Calculating estimates.F_dff')
    cnmfe_model.estimates.detrend_df_f(quantileMin=8,
                                      frames_window=250,
                                      flag_auto=False,
                                      use_residuals=False);  # use denoised data
else:
    print("estimates.F_dff already defined")

##  Deconvolution for 1p?
While we haven't discussed deconvolution (the estimation of the spikes that generated the calcium traces in `C`), we suggest treating the spike counts returned for 1p data (in `estimates.S`) with CNMFE with some caution. Currently (as of early 2024) we are aware of no simultaneous ground-truth recordings (electrophysiology + 1p imaging) that could be used to evaluate deconvolution models for 1p data. There are quite a few such recordings for 2p data (for instance see [the Spikefinder paper](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006157)).

Because of this, most researchers analyze the calcium traces directly for 1p recordings (the data in `estimates.C`) or a normalized version of them, rather than `estimates.S`.

## Save CNMF-E outputs for pipeline4onePhoton (Pipeline B)
Save following values in mat:

- `estimates`

The other NIDQ, behavior data will be merged in downstream pipeline.

In [ ]:
# =================================================================
# Save CNMF-E Output for pipeline4onePhoton (Pipeline B)
# =================================================================
# This script saves ONLY the CNMF-E results in the format expected
# by the downstream pipeline. Run this after CNMF-E processing.
#
# Output format:
#   calcium.C              - Denoised traces (n_frames x n_cells)
#   calcium.S              - Deconvolved spikes (n_frames x n_cells)
#   calcium.idx_components - Good cell indices
#   calcium.neurons_sn     - Noise per neuron
#   calcium.SNR_comp       - SNR per component
#   calcium.footprint_center - Cell centroids
#   calcium.frame_rate     - Frame rate
#   ... (other CNMF-E outputs)
# =================================================================

import numpy as np
import scipy.io
import scipy.sparse

# =================================================================
# CONFIGURATION - EDIT THESE VALUES
# =================================================================
experiment_name = 'MY01_20251231'  # EDIT: Your experiment name
SCOPE_FS = 20.0  # EDIT: Miniscope frame rate (Hz)
# =================================================================

print("=" * 80)
print(f"Saving CNMF-E output for pipeline4onePhoton-pipeline")
print(f"Experiment: {experiment_name}")
print("=" * 80)


# --- Helper function to process attributes ---
def process_attribute(name, value):
    """Process a single attribute for MAT saving."""
    # Skip private/methods
    if name.startswith('_') or callable(value):
        return None, None, False

    # Handle None
    if value is None:
        return name, np.array([]), False

    # Handle sparse matrices
    if scipy.sparse.issparse(value):
        return name, value, True

    # Handle lists/tuples
    if isinstance(value, (list, tuple)):
        try:
            return name, np.array(value), False
        except:
            return name, value, False

    # Transpose temporal components to (Frames, Neurons)
    if name in ['C', 'S', 'F_dff', 'R', 'YrA'] and isinstance(value, np.ndarray) and value.ndim == 2:
        return name, value.T, False

    # Everything else as-is
    return name, value, False


# --- STEP 1: Extract ALL attributes from estimates ---
print("\n[1/3] Extracting CNMF-E estimates...")
estimates = cnmfe_model.estimates
calcium_dict = {}
sparse_matrices = {}

all_attrs = [attr for attr in dir(estimates) if not attr.startswith('__')]
print(f"  Found {len(all_attrs)} attributes to process")

for attr_name in all_attrs:
    try:
        value = getattr(estimates, attr_name)
        proc_name, proc_value, is_sparse = process_attribute(attr_name, value)

        if proc_value is None:
            continue

        if is_sparse:
            sparse_matrices[proc_name] = proc_value
            print(f"  + {proc_name}: sparse {proc_value.shape}")
        else:
            calcium_dict[proc_name] = proc_value
            if isinstance(proc_value, np.ndarray):
                print(f"  + {proc_name}: {proc_value.shape} {proc_value.dtype}")
            else:
                print(f"  + {proc_name}: {type(proc_value).__name__}")
    except Exception as e:
        continue

# Handle sparse matrices (convert to components for MATLAB compatibility)
for sparse_name, sparse_mat in sparse_matrices.items():
    if not isinstance(sparse_mat, scipy.sparse.csc_matrix):
        sparse_mat = sparse_mat.tocsc()

    calcium_dict[f'{sparse_name}_sparse_data'] = sparse_mat.data
    calcium_dict[f'{sparse_name}_sparse_indices'] = sparse_mat.indices
    calcium_dict[f'{sparse_name}_sparse_indptr'] = sparse_mat.indptr
    calcium_dict[f'{sparse_name}_sparse_shape'] = np.array(sparse_mat.shape, dtype=np.int32)
    print(f"  + {sparse_name}: saved as sparse components")

# --- STEP 2: Add correlation image and metadata ---
print("\n[2/3] Adding metadata...")

# Add correlation image if not already present
if 'Cn' not in calcium_dict:
    calcium_dict['Cn'] = correlation_image
    print(f"  + Cn: {correlation_image.shape}")

# Add frame/neuron counts
if 'C' in calcium_dict:
    n_frames, n_neurons = calcium_dict['C'].shape
    calcium_dict['n_frames'] = int(n_frames)
    calcium_dict['n_neuron'] = int(n_neurons)
    print(f"  + n_frames: {n_frames}, n_neuron: {n_neurons}")

# Add frame rate
calcium_dict['frame_rate'] = float(SCOPE_FS)
print(f"  + frame_rate: {SCOPE_FS}")

# Add FOV size
H, W = correlation_image.shape
calcium_dict['fov_size'] = np.array([H, W], dtype=np.int32)
print(f"  + fov_size: {H} x {W}")

# --- STEP 3: Compute spatial footprints ---
print("\n[3/3] Computing spatial footprints...")
if 'A' in sparse_matrices:
    A_sparse = sparse_matrices['A']
    N_neurons = A_sparse.shape[1]

    footprints = np.zeros((N_neurons, H, W), dtype=np.float64)
    footprint_centers = np.zeros((N_neurons, 2), dtype=np.float64)

    for i in range(N_neurons):
        fp_1d = A_sparse[:, i].toarray().flatten()
        fp_2d = fp_1d.reshape(H, W)
        footprints[i] = fp_2d

        if fp_2d.sum() > 0:
            y_coords, x_coords = np.indices((H, W))
            footprint_centers[i, 0] = (y_coords * fp_2d).sum() / fp_2d.sum()
            footprint_centers[i, 1] = (x_coords * fp_2d).sum() / fp_2d.sum()
        else:
            footprint_centers[i] = [np.nan, np.nan]

    calcium_dict['footprint'] = footprints
    calcium_dict['footprint_center'] = footprint_centers
    print(f"  + Footprints computed for {N_neurons} neurons")

# --- Save to MAT file ---
print("\nSaving to MAT file...")

# Build MAT structure with ONLY calcium data
mat_data = {
    'calcium': calcium_dict,
}

mat_filename = f"cnmfe_output_{experiment_name}.mat"
scipy.io.savemat(mat_filename, mat_data, oned_as='column')

import os
file_size = os.path.getsize(mat_filename) / 1024 / 1024

print(f"\n{'=' * 80}")
print(f"CNMF-E output saved successfully!")
print(f"{'=' * 80}")
print(f"  Filename: {mat_filename}")
print(f"  Size: {file_size:.2f} MB")
print(f"  Fields in 'calcium': {len(calcium_dict)}")
print(f"\nKey fields for downstream pipeline:")
print(f"  calcium.C              : {calcium_dict['C'].shape} (denoised traces)")
print(f"  calcium.S              : {calcium_dict['S'].shape} (deconvolved spikes)")
print(f"  calcium.idx_components : {len(calcium_dict.get('idx_components', []))} good cells")
print(f"  calcium.frame_rate     : {calcium_dict['frame_rate']} Hz")
print(f"  calcium.footprint_center: {calcium_dict['footprint_center'].shape} (cell centroids)")
print(f"\nNext step:")
print(f"  1. Add this path to your config.yaml:")
print(f"     cnmfe:")
print(f"       mat_file: \"/path/to/{mat_filename}\"")
print(f"  2. Run: pipeline4onePhoton  merge --config config.yaml")
print(f"{'=' * 80}\n")


---

# Movie View (Optional)
### *Warning: Following steps requires massive memory capacity*
- Make sure you set `play_movies = True` in early cells

## View some different movie results
As with CNMF, the CNMFE model of the original movie is:

    original_movie = neural_activity + background + residual
    
The main between CNMF and CNMFE is the model of the background. We can reconstruct the neural movie as `AC` just as we did in `demo_pipeline.ipynb`. Unfortunately, reconstructing the background activity via the ring model is much more complicated for CNMFE, so we will just punt to a built-in function for that in what follows, `compute_background()`, which you can explore if you are interested:

In [ ]:
# in case you are working from loaded data, recover the raw movie
Yr, dims, num_frames = cm.load_memmap(cnmfe_model.mmap_file)
images = np.reshape(Yr.T, [num_frames] + list(dims), order='F')

Get model of neural activity and background activity (note for the neural model we are just including the accepted components):

neural_activity = cnmfe_model.estimates.A[:, cnmfe_model.estimates.idx_components] @\
                  cnmfe_model.estimates.C[cnmfe_model.estimates.idx_components, :]  # AC
neural_movie = cm.movie(neural_activity).reshape(dims + (-1,), order='F').transpose([2, 0, 1])
background_model = cnmfe_model.estimates.compute_background(Yr);  # build in function -- explore source code for details
bg_movie = cm.movie(background_model).reshape(dims + (-1,), order='F').transpose([2, 0, 1])

### View neural movie

In [ ]:
downsampling_ratio = 0.4
neural_movie.resize(fz=downsampling_ratio)
if play_movies:
    neural_movie.play(gain=1.1,
                      q_max=99.5,
                      fr=20,
                      plot_text=True,
                      magnification=2,
                      do_loop=False,
                      backend='opencv')


### View background model
You will see many regions that are constant such as blood vessels, but also lots of large-scale background flourescence, in addition to some local activity which is is on spatial scales larger than `gSig`:

In [ ]:
downsampling_ratio = 0.8
bg_movie.resize(fz=downsampling_ratio)
if play_movies:
    bg_movie.play(gain=1.1,
                  q_max=99.5,
                  fr=10,
                  plot_text=True,
                  magnification=2,
                  do_loop=False,
                  backend='opencv')

### View model and residual
We can also use the built-in `play_movie()` method to view the original movie, predicted movie, and the residual simultaneously as discussed in more detail in `demo_pipeline.ipynb`:

In [ ]:
# without background
if play_movies:
    cnmfe_model.estimates.play_movie(images,
                                     q_max=99.9,
                                     magnification=2,
                                     include_bck=False,
                                     gain_res=5,
                                     use_color=True);

A couple of things to note about that movie:
- The middle panel of neural activity includes *all* components (accepted and rejected), so you will see some of the blood vessel "activity" that was discovered and later rejected.
- The residual includes some activity that looks neural in origin. You can try playing with different params to get them (which do you think you would try?). The [Mesmerize](https://github.com/nel-lab/mesmerize-core) package is a great way to search parameter space and visualize results with more sophisticated visualization tools if you have a tricky data set.

# Clean up open resourses
Shut down server, close logger.

In [ ]:
cm.stop_server(dview=cluster)

In [ ]:
# Shut down logger (otherwise will not be able to delete it)
logging.shutdown()

In [ ]:
delete_logs = True
logging_dir = cm.paths.get_tempdir()
if delete_logs:
    log_files = glob.glob(os.path.join(logging_dir, 'demo_pipeline*.log'))
    for log_file in log_files:
        print(f"Deleting {log_file}")
        os.remove(log_file)
else:
    print(f"If you want to inspect your logs they are in {logging_dir}")